## 80. ID番号への変換
1. ~~単語をIDに変換する辞書を作成・保存(重いため,2回目以降はセル2を実行)~~
2. 辞書の読み込み
3. 単語列をID列に変換する関数

In [4]:
# pklファイルから読み込み
import pickle

with open('../data/ch09/name_to_id.pkl', 'rb') as tf:
    name_to_id = pickle.load(tf)

In [5]:
# 与えられた単語列に対し, ID番号の列を返す関数
# ch06ではCountVectorizerを利用したため, 今回もCountVectorizerを活用する
from sklearn.feature_extraction.text import CountVectorizer

def convert_words_to_ids(words):
    '''
    input :words(単語列)
    output:ids(ID番号列)
    '''
    # analyzer: 単語列に前処理を加え, listに変換する関数
    analyzer = CountVectorizer().build_analyzer()
    word_list = analyzer(words)
    
    ids = []
    for word in word_list:
        if word in name_to_id:
            ids.append(name_to_id[word])
        else:
            ids.append(0)  # 未知語の場合, IDを0とする
    
    return ids

## GPU prepare
1. 使用可能GPUの確認
2. GPUの指定
3. PyTorchで利用できるGPU数の確認

In [3]:
# 使用可能GPUの確認
!nvidia-smi

Tue Jun 14 19:25:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8    N/A /  N/A |     78MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# GPUの指定
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' #0番を使用するとき

In [5]:
# 確認
import torch
print(torch.cuda.device_count()) #Pytorchで使用できるGPU数を取得

1


## prepare
1. 語彙数の取得
2. 学習データの用意(ラベル)
3. 学習データの用意(特徴量)
4. 乱数の種を固定

In [6]:
# 語彙数の取得(ID:0の単語はまとめて1語とする), 未知語, paddingを考慮
vocab_size = max(name_to_id.values())+2

In [7]:
# 訓練・検証・評価データの用意
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# ラベル: ch08の出力を利用
Y_train = np.loadtxt('../data/ch08/Y_train.txt')
Y_valid = np.loadtxt('../data/ch08/Y_valid.txt')
Y_test = np.loadtxt('../data/ch08/Y_test.txt')

# pytorch用に変換
Y_train_long = torch.tensor(Y_train, dtype=torch.int64)
Y_valid_long = torch.tensor(Y_valid, dtype=torch.int64)
Y_test_long = torch.tensor(Y_test, dtype=torch.int64)

In [8]:
# 特徴量: convert_words_to_ids(80)を利用
def convert_text_to_features(fname):
    '''
    input :fname
    output:features(tensor)
    '''
    with open(fname, encoding='utf-8') as f:
        lines = f.readlines()
    
    # id列(list)のリストに変換
    ids_list = [convert_words_to_ids(line) for line in lines]
    
    # id列(tensor)のリストに変換
    ids_tensor = [torch.tensor(ids, dtype=torch.int64) for ids in ids_list]
    
    # 最大のid+1(vocab_size-1)でパディング
    features = torch.nn.utils.rnn.pad_sequence(ids_tensor, batch_first=True, padding_value=vocab_size-1)
    
    return features

# 特徴量抽出
X_train_long = convert_text_to_features('../data/ch06/train.txt')
X_valid_long = convert_text_to_features('../data/ch06/valid.txt')
X_test_long = convert_text_to_features('../data/ch06/test.txt')

In [9]:
# 乱数シードの固定
import random

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

## 83. ミニバッチ化・GPU上での学習
1. ~~GPUに対応したRNNの定義~~
2. GPUに対応したaccuracy計測関数の定義
3. ~~ミニバッチ化し, GPU上で学習~~

In [10]:
# accuracyの計測
def measure_loss_accuracy2(model, criterion, dataloader):
    '''
    input : model, criterion, dataloader
    output: loss, accuracy
    '''
    running_loss = 0
    correct_num = 0
    device = model.device
    batch_size = model.batch_size
    for X, Y in dataloader:
        # GPU上に
        X = X.to(device)
        Y = Y.to(device)
        model.init_hidden()
        predict_y = model.forward(X)
        
        # lossの計算
        loss = criterion(predict_y, Y)
        running_loss += loss.item()
        
        # accuracyの計算
        predict_label = torch.max(predict_y, 1)[1]
        for i in range(batch_size):
            if predict_label[i] == Y[i]:
                correct_num += 1
    
    loss_avg = running_loss/len(dataloader)
    accuracy = correct_num/(len(dataloader)*batch_size)
    
    return loss_avg, accuracy

## 84. 単語ベクトルの導入
1. GoogleNewsデータセットから使用する単語を抜き出し, 埋め込みを保存(2回目以降はセル2から実行)
2. 学習済み単語ベクトルでembを初期化したRNNの定義
3. ミニバッチ化し, GPU上で学習

In [10]:
'''
# GoogleNewsデータセット
import gensim

google_news_all = gensim.models.KeyedVectors.load_word2vec_format('../data/ch07/GoogleNews-vectors-negative300.bin', binary=True)

# 埋め込みの作成 size: (vocab_size, 300)
# [-1:1]の一様分布で初期化 (事前学習済みベクトルの範囲が凡そ[-1:1]であるため)
fix_seed(42)
embedding_inital = np.random.rand(vocab_size, 300) * 2 - 1

# paddingに対応する箇所を0にする
embedding_inital[vocab_size-1] = [0] * 300

# GoogleNewsデータセット内にあるものを事前学習済み単語ベクトルで初期化
for name, id in name_to_id.items():
    if (id!=0) & (name in google_news_all):
        embedding_inital[id] = google_news_all[name]

# ファイルに保存
np.savetxt('../data/ch09/84_embedding_initial.txt', embedding_inital)
'''

In [11]:
# RNNの定義
class RNN3(nn.Module):
    def __init__(self, embedding_inital, hidden_size, output_size, batch_size, num_layers=1):
        # 層の定義
        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        vocab_size = embedding_inital.shape[0]
        embedding_dim = embedding_inital.shape[1]
        
        self.emb = nn.Embedding.from_pretrained(embedding_inital, freeze=False, padding_idx=vocab_size-1)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True, num_layers=self.num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

        # GPUに移す
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self = self.to(self.device)
    
    def init_hidden(self, batch_size=None):
        if not batch_size:
            batch_size = self.batch_size
        self.hidden_state = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
    
    def forward(self, x):
        x = self.emb(x)
        x_rnn, self.hidden_state =self.rnn(x, self.hidden_state)
        x = self.fc(x_rnn[:,-1,:])
        return x

In [14]:
import torch.optim as optim

# 埋め込みの読み込み
# embedding_initialを5倍する
#   embedding_initialの元の値の範囲が凡そ[-1.0,1.0]
#   rnn2.emb.weightの初期値の範囲が凡そ[-5.0, 5.0]
#   => 範囲をそろえるため, 全体を5倍
embedding_inital = np.loadtxt('../data/ch09/84_embedding_initial.txt') * 5
embedding_inital = torch.from_numpy(embedding_inital.astype(np.float32)).clone()

# ハイパーパラメータの設定
embedding_dim = 300
hidden_size = 50
max_epoch = 25
batch_size = 8
lr = 0.002

# dataloaderの定義
train_dataset2 = torch.utils.data.TensorDataset(X_train_long, Y_train_long)
train_dataloader2 = torch.utils.data.DataLoader(train_dataset2, batch_size=batch_size, drop_last=True)

valid_dataset2 = torch.utils.data.TensorDataset(X_valid_long, Y_valid_long)
valid_dataloader2 = torch.utils.data.DataLoader(valid_dataset2, batch_size=batch_size, drop_last=True)

# モデルの定義
fix_seed(42)
rnn3 = RNN3(embedding_inital=embedding_inital, hidden_size=hidden_size, output_size=4, batch_size=batch_size)
criterion3 = nn.CrossEntropyLoss()
optimizer3 = optim.SGD(rnn3.parameters(), lr)
device = rnn3.device

# SGDによる学習
for epoch in range(max_epoch):
    running_loss = 0
    for X, Y in train_dataloader2:
        # GPU上に
        X = X.to(device)
        Y = Y.to(device)
        # forward
        rnn3.init_hidden()
        predict_y = rnn3.forward(X)
        loss = criterion3(predict_y, Y)
        
        # backward
        optimizer3.zero_grad()
        loss.backward()
        
        # 更新
        optimizer3.step()
        running_loss += loss.item()
    
    # loss, accuracyの表示
    rnn3.init_hidden()
    valid_loss, valid_acc = measure_loss_accuracy2(rnn3, criterion3, valid_dataloader2)
    train_loss, train_acc = measure_loss_accuracy2(rnn3, criterion3, train_dataloader2)
    train_loss = running_loss/len(train_dataloader2)
    
    print(f'epoch: {epoch}')
    print(f'   train loss: {train_loss}\ttrain acc: {train_acc}')
    print(f'   valid loss: {valid_loss}\tvalid acc: {valid_acc}')

epoch: 0
   train loss: 1.2115005899904372	train acc: 0.4207865168539326
   valid loss: 1.1631733264751778	valid acc: 0.4184131736526946
epoch: 1
   train loss: 1.161489953575063	train acc: 0.4206928838951311
   valid loss: 1.1615557145929623	valid acc: 0.42065868263473055
epoch: 2
   train loss: 1.1606642708349764	train acc: 0.42312734082397
   valid loss: 1.1602516324220302	valid acc: 0.4281437125748503
epoch: 3
   train loss: 1.1596257258443796	train acc: 0.4287453183520599
   valid loss: 1.1584172273824316	valid acc: 0.4408682634730539
epoch: 4
   train loss: 1.1579188249084387	train acc: 0.4444756554307116
   valid loss: 1.1552287729200488	valid acc: 0.47005988023952094
epoch: 5
   train loss: 1.1541319531001402	train acc: 0.47752808988764045
   valid loss: 1.1475939764947949	valid acc: 0.49176646706586824
epoch: 6
   train loss: 1.1359927204217803	train acc: 0.5661985018726592
   valid loss: 1.097729177888996	valid acc: 0.5568862275449101
epoch: 7
   train loss: 0.962642435299769